In [2]:
%load_ext autoreload
%autoreload 2

from sympy import *
from sympy.tensor import *
import sympy.printing as printing
from sympy.physics.vector import dynamicsymbols

from variables import *
from structure import *
from functions import *
from latex import *

import utils
from utils import write_obj, read_obj, c_print
from utils import print_all_variables as pa
# from main import SymbolSolver


Matrix([
[-sin(alpha(t))*cos(theta0(t)) - sin(theta0(t))*cos(alpha(t))],
[-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t))],
[                                                           0]])

In [4]:
# ss = SymbolSolver()
# ss.get_omegas(calc=True)
# ss.get_euler_equations(calc=True)

calc=True

# угловые скорости (платформы, вилки, колеса) относительно пола

In [5]:
if calc:
    omega['platform']= lambda i: Derivative(alpha,t)*e['z']
    omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
    omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)
    write_obj(omega['platform'], 'angular_velocity_platform', 'угловая скорость платформы')
    write_obj(omega['fork'], 'angular_velocity_fork', 'угловая скорость вилки')
    write_obj(omega['wheel'], 'angular_velocity_wheel', 'угловая скорость колеса')
else:
    omega['platform'] = read_obj('angular_velocity_platform')
    omega['fork'] = read_obj('angular_velocity_fork')
    omega['wheel'] = read_obj('angular_velocity_wheel')

In [6]:
cos(beta[0]) * e['xi'] + sin(beta[0])*e['eta']

Matrix([
[-sin(alpha(t))*sin(beta0(t)) + cos(alpha(t))*cos(beta0(t))],
[ sin(alpha(t))*cos(beta0(t)) + sin(beta0(t))*cos(alpha(t))],
[                                                         0]])

# УРАНЕНИЯ ЭЁЛЕРА И ОТСУТСТВИЕ ПРОСКАЛЬЗЫВАНИЯ 


In [7]:
if calc:
    v['S'] = euler(S, P)
    v['P'] = euler(P, C)
    v['C'] = euler(C, D)
    v['D'] = lambda i: Matrix([0,0,0]) # проскальзывания нет
    
    write_obj(v['S'], 'point_S_velocity', 'скорость точки S')
    write_obj(v['P'], 'point_P_velocity', 'скорость точки P')
    write_obj(v['C'], 'point_C_velocity', 'скорость точки C')
    write_obj(v['D'], 'point_D_velocity', 'скорость точки D')
else:
    v['S'] = read_obj('point_S_velocity')
    v['P'] = read_obj('point_P_velocity')
    v['C'] = read_obj('point_C_velocity')
    v['D'] = read_obj('point_D_velocity')

In [8]:
simplify(v['P'](0))

Matrix([
[ d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) + r*cos(alpha(t) + theta0(t))*Derivative(psi0(t), t)],
[-d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + r*sin(alpha(t) + theta0(t))*Derivative(psi0(t), t)],
[                                                                                                                                     0]])

In [9]:
latex(simplify(v['S'](0))[2])

'0'

In [10]:
# init_printing()
# simplify(Transpose(v['C'](0))*v['C'](0))

In [ ]:
# eq['nu1'](0)

# Полученные выражения из связей для nu1 nu2 delta_x delta_y

In [11]:
if calc:
    eq['delta_x'] = lambda i: scalar(v['S'](i), e['x'])
    eq['delta_y'] = lambda i: scalar(v['S'](i), e['y'])
    eq['nu1']     = lambda i: scalar(v['S'](i), e['xi'])
    eq['nu2']     = lambda i: scalar(v['S'](i), e['eta'])
    write_obj(eq['delta_x'], 'delta_x', '')
    write_obj(eq['delta_y'], 'delta_x', '')
    write_obj(eq['nu1'], 'nu1', '')
    write_obj(eq['nu2'], 'nu2', '')
else:
    read_obj('delta_x')
    read_obj('delta_x')
    read_obj('nu1')
    read_obj('nu2')

In [12]:
latex(simplify(eq['nu2'](0)))

'- d \\cos{\\left (\\theta_{0}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\alpha{\\left (t \\right )} - d \\cos{\\left (\\theta_{0}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\theta_{0}{\\left (t \\right )} + r \\sin{\\left (\\theta_{0}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\psi_{0}{\\left (t \\right )} - \\cos{\\left (\\beta_{0}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\alpha{\\left (t \\right )}'

# Связи через delta_alpha, delta_x, delta_y

In [13]:
if calc:
    # возвращает словарь с выражениями для diff(psi) и diff(theta)
    eq['f(delta_x,delta_y)'] = lambda i: solve(
                          [Eq(eq['delta_x'](i), delta['x']), Eq(eq['delta_y'](i), delta['y'])],
                          [Derivative(psi[i],t), Derivative(theta[i],t)],
                          dict=True)[0];


    # !!! delta_x delta_y - symbols
    eq['diff(psi)']   = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(psi[i],t)].\
        subs(Derivative(alpha,t), delta['alpha']).subs(Derivative(psi[i],t), delta['psi']).\
        subs(Derivative(theta[i],t), delta['theta'])
    eq['diff(theta)'] = lambda i: eq['f(delta_x,delta_y)'](i)[Derivative(theta[i],t)].\
        subs(Derivative(alpha,t), delta['alpha']).subs(Derivative(psi[i],t), delta['psi']).\
        subs(Derivative(theta[i],t), delta['theta'])

    write_obj(eq['diff(psi)'], 'diff(psi)', '3 уравнения связей на diff(psi) = ...')
    write_obj(eq['diff(theta)'], 'diff(theta)', '3 уравнения связей на diff(theta) = ...')
else:
    eq['diff(psi)'] = read_obj('diff(psi)')
    eq['diff(theta)'] = read_obj('diff(theta)')

In [14]:
latex(eq['diff(psi)'](0))

'\\frac{- \\delta_{\\alpha} \\sin{\\left (\\beta_{0}{\\left (t \\right )} - \\theta_{0}{\\left (t \\right )} \\right )} + \\delta_{x} \\cos{\\left (\\alpha{\\left (t \\right )} + \\theta_{0}{\\left (t \\right )} \\right )} + \\delta_{y} \\sin{\\left (\\alpha{\\left (t \\right )} + \\theta_{0}{\\left (t \\right )} \\right )}}{r}'

# Связи через nu_1, nu_2

In [15]:
if calc:
    eq['f(nu1,nu2)'] = lambda i: solve(
                          [Eq(eq['nu1'](i), nu[1]), Eq(eq['nu2'](i), nu[2])],
                          [Derivative(psi[i],t), Derivative(theta[i],t)],
                          dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

    eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
    eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]
    
    write_obj(eq['diff(psi)_nu'], 'diff(psi)_nu', '3 уравнения связей на diff(psi) = ..., только уже от nu1 и nu2')
    write_obj(eq['diff(theta)_nu'], 'diff(theta)_nu', '3 уравнения связей на diff(theta) = ..., только уже от nu1 и nu2')
else:
    eq['diff(psi)_nu'] = read_obj('diff(psi)_nu')
    eq['diff(theta)_nu'] = read_obj('diff(theta)_nu')

In [16]:
latex(eq['diff(theta)_nu'](0))

'\\frac{- d \\frac{d}{d t} \\alpha{\\left (t \\right )} + \\nu_{1}{\\left (t \\right )} \\sin{\\left (\\theta_{0}{\\left (t \\right )} \\right )} - \\nu_{2}{\\left (t \\right )} \\cos{\\left (\\theta_{0}{\\left (t \\right )} \\right )} - \\cos{\\left (\\beta_{0}{\\left (t \\right )} - \\theta_{0}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\alpha{\\left (t \\right )}}{d}'

# ТЕЛО 1 (платформа)

In [17]:
velocity['S']            = lambda i: nu[1]*e['xi'] + nu[2]*e['eta']
F['platform']            = lambda i: zeros(3,1) # сил не действует
delta_r['S']             = subs_delta(lambda i: delta['x']*e['x'] + delta['y']*e['y'])
J['platform']            = eye(3,3)*a
K['platform']            = lambda i: J['platform']*omega['platform'](i)
omega_delta['platform']  = subs_delta(lambda i: omega['platform'](i).\
                                      subs(Derivative(alpha,t),delta['alpha'])) # alpha psi theta
M['platform']            = lambda i: -(-W[i + 1]*e['z']) # +1 потому что W[1], W[2], W[3]

A['platform'] = dalamber(mass = m['platform'],
                         velocity = velocity['S'],
                         F = F['platform'],
                         delta_r = delta_r['S'],
                         K = K['platform'],
                         M = M['platform'],
                         omega_delta = omega_delta['platform'])

# ТЕЛО 2 (вилки)

In [18]:
velocity['P']        = v['P']
F['fork']            = lambda i: zeros(3,1) # сил не действует
delta_r['P']         = subs_delta(lambda i: delta_r['S'](i) + cross(omega['platform'](i), vec_by_2dots(S,P)(i)))
J['fork']            = zeros(3,3)
K['fork']            = lambda i: J['fork']*omega['fork'](i)
omega_delta['fork']  = subs_delta(lambda i: omega['fork'](i))
M['fork']            = lambda i: -(W[i + 1]*e['z']  - T[i + 1]*n_wheel(i))

A['fork'] = dalamber(mass = 0,
                     velocity = velocity['P'],
                     F = F['fork'],
                     delta_r = delta_r['P'],
                     K = K['fork'],
                     M = M['fork'],
                     omega_delta = omega_delta['fork'])

# ТЕЛО 3 (колёса)

In [41]:
velocity['C']         = v['C']
F['wheel']            = lambda i: zeros(3,1) # сил не действует
delta_r['C']          = subs_delta(lambda i: delta_r['P'](i) + cross(omega['fork'](i), vec_by_2dots(P,C)(i)))
J['wheel']            = Matrix([[b,0,0],[0,c,0],[0,0,b]])
# K['wheel']            = lambda i: J['wheel'] * Matrix([scalar(omega['wheel'](i), e_wheel(i)), 
#                                                        scalar(omega['wheel'](i), n_wheel(i)),
#                                                        scalar(omega['wheel'](i), e['z'])])
K['wheel']            = lambda i: c*psi[i].diff(t)*n_wheel(i) + b*(alpha.diff(t) + theta[0].diff(t))*e['z']
omega_delta['wheel']  = subs_delta(lambda i: omega['wheel'](i))
M['wheel']            = lambda i: -T[i + 1]*n_wheel(i)

A['wheel'] = dalamber(mass= m['wheel'],
                     velocity = velocity['C'],
                     F = F['wheel'],
                     delta_r = delta_r['C'],
                     K = K['wheel'],
                     M = M['wheel'],
                     omega_delta = omega_delta['wheel'])


In [40]:
K['wheel']            = lambda i: J['wheel'] * Matrix([scalar(omega['wheel'](i), e_wheel(i)), 
                                                       scalar(omega['wheel'](i), n_wheel(i)),
                                                       scalar(omega['wheel'](i), e['z'])])
simplify(K['wheel'](0))

Matrix([
[                                                     0],
[                              c*Derivative(psi0(t), t)],
[b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))]])

In [21]:
simplify(c*delta['psi'][0]*n_wheel(0) + b*(alpha.diff(t) + theta[0].diff(t))*e['z'])

Matrix([
[            -c*delta_psi0(t)*sin(alpha(t) + theta0(t))],
[             c*delta_psi0(t)*cos(alpha(t) + theta0(t))],
[b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))]])

In [60]:
print(latex(simplify(K['wheel'](0))))

\left[\begin{matrix}- c \sin{\left (\alpha{\left (t \right )} + \theta_{0}{\left (t \right )} \right )} \frac{d}{d t} \psi_{0}{\left (t \right )}\\c \cos{\left (\alpha{\left (t \right )} + \theta_{0}{\left (t \right )} \right )} \frac{d}{d t} \psi_{0}{\left (t \right )}\\b \left(\frac{d}{d t} \alpha{\left (t \right )} + \frac{d}{d t} \theta_{0}{\left (t \right )}\right)\end{matrix}\right]


In [57]:
K['wheel'](0).diff(t)

Matrix([
[ c*(-sin(alpha(t))*cos(theta0(t)) - sin(theta0(t))*cos(alpha(t)))*Derivative(psi0(t), (t, 2)) + c*(sin(alpha(t))*sin(theta0(t))*Derivative(alpha(t), t) + sin(alpha(t))*sin(theta0(t))*Derivative(theta0(t), t) - cos(alpha(t))*cos(theta0(t))*Derivative(alpha(t), t) - cos(alpha(t))*cos(theta0(t))*Derivative(theta0(t), t))*Derivative(psi0(t), t)],
[c*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*Derivative(psi0(t), (t, 2)) + c*(-sin(alpha(t))*cos(theta0(t))*Derivative(alpha(t), t) - sin(alpha(t))*cos(theta0(t))*Derivative(theta0(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(alpha(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(theta0(t), t))*Derivative(psi0(t), t)],
[                                                                                                                                                                                                                                                                                      b*(Derivative(alpha(

In [82]:
#
#
# scalar(velocity['P'](0), delta_r['P'](0)) + scalar(M['fork'](0), omega_delta['fork'](0))

In [25]:
# A['wheel'] = dalamber_subs(A['wheel'])
# A['fork'] = dalamber_subs(A['wheel'])
# A['platfrom'] = dalamber_subs(A['wheel'])

# Весь Даламбер / для одного колеса

In [42]:
# I have not the foggiest idea
# time ~1.5min

# all_dalamber = dalamber_subs(A['wheel'])(0)
# -> all_dalamber += dalamber_subs(A['fork'])(i) + dalamber_subs(A['platform'])(i)

if calc:
    all_dalamber = dalamber_subs(A['platform'], eq)(0)
    for i in range(1): # для одного
        all_dalamber += dalamber_subs(A['fork'], eq)(i) + dalamber_subs(A['wheel'], eq)(i)

        write_obj(all_dalamber, 'all_dalamber', 'Даламбер для всего')
else:
    all_dalamber = read_obj('all_dalamber')

In [43]:
all_dalamber

b*(delta_alpha + (-d*delta_alpha - delta_alpha*cos(beta0(t) - theta0(t)) + delta_x*sin(alpha(t) + theta0(t)) - delta_y*cos(alpha(t) + theta0(t)))/d)*(Derivative(alpha(t), (t, 2)) + Derivative(theta0(t), (t, 2))) + delta_alpha*(a*Derivative(alpha(t), (t, 2)) - W1(t)) + delta_x*m1*(-nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) - sin(alpha(t))*Derivative(nu2(t), t) + cos(alpha(t))*Derivative(nu1(t), t)) + delta_y*m1*(nu1(t)*cos(alpha(t))*Derivative(alpha(t), t) - nu2(t)*sin(alpha(t))*Derivative(alpha(t), t) + sin(alpha(t))*Derivative(nu1(t), t) + cos(alpha(t))*Derivative(nu2(t), t)) + m2*(r*(-sin(alpha(t))*sin(theta0(t)) + cos(alpha(t))*cos(theta0(t)))*Derivative(psi0(t), (t, 2)) + r*(-sin(alpha(t))*cos(theta0(t))*Derivative(alpha(t), t) - sin(alpha(t))*cos(theta0(t))*Derivative(theta0(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(alpha(t), t) - sin(theta0(t))*cos(alpha(t))*Derivative(theta0(t), t))*Derivative(psi0(t), t))*(-d*(delta_alpha

In [44]:
if calc:
    poly_dalamber = Poly(all_dalamber, [delta['alpha'], delta['x'], delta['y']])
    write_obj(poly_dalamber, 'poly_dalamber', 'Даламбер для всего, полином')
else:
    read_obj('poly_dalamber')

In [76]:
# xtemp, ytemp = symbols('xtemp, ytemp')
# Poly(xtemp*3+ytemp*2 + 3, [xtemp, ytemp]).coeffs()[1]

In [32]:
# test = poly_dalamber.coeff_monomial(delta['alpha']) + poly_dalamber.coeff_monomial(delta['x']) + poly_dalamber.coeff_monomial(delta['y'])

In [105]:
temp = all_dalamber - s_dalpha*delta['alpha'] - s_dx*delta['x'] - s_dy*delta['y']

In [45]:
temp = all_dalamber.diff(delta['y']) - s_dy
simplify(temp)

NameError: name 's_dy' is not defined

In [46]:
if calc:
    s_dalpha = simplify(poly_dalamber.coeff_monomial(delta['alpha']))
    s_dx = simplify(poly_dalamber.coeff_monomial(delta['x']))
    s_dy = simplify(poly_dalamber.coeff_monomial(delta['y']))
    
    write_obj(s_dx, 'dalamber_dx_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dx')
    write_obj(s_dy, 'dalamber_dy_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dy')
    write_obj(s_dalpha, 'dalamber_dalpha_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dAlpha')
else:
    read_obj('dalamber_dx_coeff')
    read_obj('dalamber_dy_coeff')
    read_obj('dalamber_dalpha_coeff')

In [47]:
if calc:
    TWA_s = Matrix([
        [s_dalpha.diff(T[1]), s_dalpha.diff(W[1]), s_dalpha.diff(Derivative(alpha, t, 2))],
        [s_dx.diff(T[1]),     s_dx.diff(W[1]),     s_dx.diff(Derivative(alpha, t, 2))],
        [s_dy.diff(T[1]),     s_dy.diff(W[1]),     s_dy.diff(Derivative(alpha, t, 2))]
    ])
    write_obj(TWA_s, 'TWA', "Матрица относительно T, W, alpha'' ")
else:
    read_obj('TWA')

In [52]:
ns_dalpha = poly_dalamber.coeff_monomial(delta['alpha'])
ns_dx = poly_dalamber.coeff_monomial(delta['x'])
ns_dy = poly_dalamber.coeff_monomial(delta['y'])

In [53]:
TWA = Matrix([
    [ns_dalpha.diff(T[1]), ns_dalpha.diff(W[1]), ns_dalpha.diff(Derivative(alpha, t, 2))],
    [ns_dx.diff(T[1]),     ns_dx.diff(W[1]),     ns_dx.diff(Derivative(alpha, t, 2))],
    [ns_dy.diff(T[1]),     ns_dy.diff(W[1]),     ns_dy.diff(Derivative(alpha, t, 2))]
])

In [51]:
simplify(TWA_s.det())

-(a + b)/(d*r)

In [54]:
simplify(TWA.det())

-(a + b)/(d*r)

In [67]:
TWA_s

Matrix([
[-sin(beta0(t) - theta0(t))/r, (-d*r - r*cos(beta0(t) - theta0(t)))/(d*r), (a*d*r - b*r*cos(beta0(t) - theta0(t)))/(d*r)],
[ cos(alpha(t) + theta0(t))/r,                sin(alpha(t) + theta0(t))/d,                 b*sin(alpha(t) + theta0(t))/d],
[ sin(alpha(t) + theta0(t))/r,               -cos(alpha(t) + theta0(t))/d,                -b*cos(alpha(t) + theta0(t))/d]])

In [89]:
TWA_right = Matrix([Poly(s_dalpha, [W[1], T[1], Derivative(alpha, t, 2)]).coeffs()[3],
            Poly(s_dx, [W[1], T[1], Derivative(alpha, t, 2)]).coeffs()[3],
            Poly(s_dy, [W[1], T[1], Derivative(alpha, t, 2)]).coeffs()[3]])

In [94]:
TWA_right_s = simplify(TWA_right)

In [ ]:
eq['nu1'] = cos(alpha)*diff(x) + sin(alpha)*diff(y)
eq['nu2'] = -sin(alpha)*diff(x) + cos(alpha)*diff(y)

In [98]:
TWA_res = simplify(TWA_s.inv()) * TWA_right_s

In [100]:
TWA_res_s = simplify(TWA_res)

In [106]:
# s_dalpha = simplify(poly_dalamber.coeff_monomial(delta['alpha']))

In [107]:
# s_dx = simplify(poly_dalamber.coeff_monomial(delta['x']))

In [108]:
# s_dy = simplify(poly_dalamber.coeff_monomial(delta['y']))

In [109]:
# write_obj(s_dx, 'dalamber_dx_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dx')
# write_obj(s_dy, 'dalamber_dy_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dy')
# write_obj(s_dalpha, 'dalamber_dalpha_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dAlpha')

In [129]:
# eq['diff(psi)_nu'](0)

-(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + sin(beta0(t) - theta0(t))*Derivative(alpha(t), t))/r

In [64]:
calc=False

In [141]:
# dalamber_subs_nu(Derivative(psi[0], t, 2), eq)

Derivative(-(nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)) + sin(beta0(t) - theta0(t))*Derivative(alpha(t), t))/r, t)

In [65]:
if calc:
    s_dalpha_nu = dalamber_subs_nu(s_dalpha, eq)
    s_dx_nu = dalamber_subs_nu(s_dx, eq)
    s_dy_nu = dalamber_subs_nu(s_dy, eq)
    
    write_obj(s_dx, 'dalamber_dx_coeff_nu', 'коэффициент в уравнении Даламбера-Лагранжа при dx')
    write_obj(s_dy, 'dalamber_dy_coeff_nu', 'коэффициент в уравнении Даламбера-Лагранжа при dy')
    write_obj(s_dalpha, 'dalamber_dalpha_coeff_nu', 'коэффициент в уравнении Даламбера-Лагранжа при dAlpha')
else:
    s_dx_nu = read_obj('dalamber_dx_coeff_nu')
    s_dy_nu = read_obj('dalamber_dy_coeff_nu')
    s_dalpha_nu = read_obj('dalamber_dalpha_coeff_nu')

In [66]:
s_dx_nu

(d*r*(-m1*nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - m1*nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) - m1*sin(alpha(t))*Derivative(nu2(t), t) + m1*cos(alpha(t))*Derivative(nu1(t), t) + m2*r*sin(alpha(t) + theta0(t))*Derivative(alpha(t), t)*Derivative(psi0(t), t) + m2*r*sin(alpha(t) + theta0(t))*Derivative(psi0(t), t)*Derivative(theta0(t), t) + m2*r*sin(alpha(t) + theta1(t))*Derivative(alpha(t), t)*Derivative(psi1(t), t) + m2*r*sin(alpha(t) + theta1(t))*Derivative(psi1(t), t)*Derivative(theta1(t), t) + m2*r*sin(alpha(t) + theta2(t))*Derivative(alpha(t), t)*Derivative(psi2(t), t) + m2*r*sin(alpha(t) + theta2(t))*Derivative(psi2(t), t)*Derivative(theta2(t), t) - m2*r*cos(alpha(t) + theta0(t))*Derivative(psi0(t), (t, 2)) - m2*r*cos(alpha(t) + theta1(t))*Derivative(psi1(t), (t, 2)) - m2*r*cos(alpha(t) + theta2(t))*Derivative(psi2(t), (t, 2))) + d*(c*sin(alpha(t))*sin(theta0(t))*cos(alpha(t) + theta0(t))*Derivative(psi0(t), (t, 2)) + c*sin(alpha(t))*sin(theta1(t))*cos(alpha(t) + theta1(t

In [151]:
simplify(eq['diff(psi)_nu'](0).diff(t))

(-(Derivative(beta0(t), t) - Derivative(theta0(t), t))*cos(beta0(t) - theta0(t))*Derivative(alpha(t), t) + nu1(t)*sin(theta0(t))*Derivative(theta0(t), t) - nu2(t)*cos(theta0(t))*Derivative(theta0(t), t) - sin(beta0(t) - theta0(t))*Derivative(alpha(t), (t, 2)) - sin(theta0(t))*Derivative(nu2(t), t) - cos(theta0(t))*Derivative(nu1(t), t))/r

In [162]:
eq['diff(psi)_nu'](0).subs(nu[2], 0).subs(nu[1], a*t).subs(Derivative(alpha, t), 0)

-a*t*cos(theta0(t))/r

In [163]:
eq['diff(theta)_nu'](0).subs(nu[2], 0).subs(nu[1], a*t).subs(Derivative(alpha, t), 0)

-a*t*sin(theta0(t))/d

In [191]:
init_printing(use_latex='mathjax')
eq['diff(theta)_nu'](0)

    d                                                                   d     
- d⋅──(α(t)) - ν₁(t)⋅sin(θ₀(t)) + ν₂(t)⋅cos(θ₀(t)) - cos(β₀(t) - θ₀(t))⋅──(α(t
    dt                                                                  dt    
──────────────────────────────────────────────────────────────────────────────
                                       d                                      

  
))
  
──
  

In [61]:
m['platform']

m1

(W1*delta_x*sin(alpha(t) + theta[0])/d + W2*delta_x*sin(alpha(t) + theta[1])/d + W3*delta_x*sin(alpha(t) + theta[2])/d - b*delta_x*sin(alpha(t) + theta[0])*Derivative(alpha(t), (t, 2))/d - b*delta_x*sin(alpha(t) + theta[1])*Derivative(alpha(t), (t, 2))/d - b*delta_x*sin(alpha(t) + theta[2])*Derivative(alpha(t), (t, 2))/d + 3*delta_x*m1*(-nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) - sin(alpha(t))*Derivative(nu2(t), t) + cos(alpha(t))*Derivative(nu1(t), t)) + delta_x*(-T1*(-sin(alpha(t))*sin(theta[0]) + cos(alpha(t))*cos(theta[0])) + c*((-sin(alpha(t))*sin(theta[0]) + cos(alpha(t))*cos(theta[0]))*(-2*sin(alpha(t))*cos(theta[0])*Derivative(alpha(t), t) - 2*sin(theta[0])*cos(alpha(t))*Derivative(alpha(t), t))*Derivative(psi[0], t) + (-sin(alpha(t))*cos(theta[0]) - sin(theta[0])*cos(alpha(t)))*(2*sin(alpha(t))*sin(theta[0])*Derivative(alpha(t), t) - 2*cos(alpha(t))*cos(theta[0])*Derivative(alpha(t), t))*Derivative(psi[0], t)))*(-sin(alpha(t))

In [63]:
def diff_vars(obj1, obj2):
    res = []
    for x in pa(obj1):
        if x not in pa(obj2):
            res.append(x)
    return res

[dalpha] [psi[1], c, psi[0], psi, Derivative(psi[2], t), Derivative(psi[1], t), delta_alpha, Derivative(psi[0], t), psi[2]]
[dx] [psi[1], c, psi[0], psi, Derivative(psi[2], t), Derivative(psi[1], t), Derivative(psi[0], t), psi[2]]
[dy] [psi[1], c, psi[0], psi, Derivative(psi[2], t), Derivative(psi[1], t), Derivative(psi[0], t), psi[2]]


NameError: name 'poly_dalamber' is not defined

In [65]:
s_dx = simplify(delta_x_k)

In [66]:
s_dy = simplify(delta_y_k)

In [67]:
s_dalpha = simplify(delta_alpha_k)

In [73]:
utils.write_obj(s_dx, 'dalamber_dx_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dx')
utils.write_obj(s_dy, 'dalamber_dy_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dy')
utils.write_obj(s_dalpha, 'dalamber_dalpha_coeff', 'коэффициент в уравнении Даламбера-Лагранжа при dAlpha')

In [60]:
# poly_dalamber.coeff_monomial(delta['alpha']) + poly_dalamber.coeff_monomial(delta['x']) + poly_dalamber.coeff_monomial(delta['y'])

-T1*sin(alpha(t) + theta[0])*sin(alpha(t))**2*sin(theta[0])**2/r - T1*sin(alpha(t) + theta[0])*sin(alpha(t))**2*cos(theta[0])**2/r - T1*sin(alpha(t) + theta[0])*sin(theta[0])**2*cos(alpha(t))**2/r - T1*sin(alpha(t) + theta[0])*cos(alpha(t))**2*cos(theta[0])**2/r - T1*sin(beta[0] - theta[0])*sin(alpha(t))**2*sin(theta[0])**2/r - T1*sin(beta[0] - theta[0])*sin(alpha(t))**2*cos(theta[0])**2/r - T1*sin(beta[0] - theta[0])*sin(theta[0])**2*cos(alpha(t))**2/r - T1*sin(beta[0] - theta[0])*cos(alpha(t))**2*cos(theta[0])**2/r - T1*sin(alpha(t))**2*sin(theta[0])**2*cos(alpha(t) + theta[0])/r - T1*sin(alpha(t))**2*cos(alpha(t) + theta[0])*cos(theta[0])**2/r - T1*sin(theta[0])**2*cos(alpha(t) + theta[0])*cos(alpha(t))**2/r - T1*cos(alpha(t) + theta[0])*cos(alpha(t))**2*cos(theta[0])**2/r - T2*sin(alpha(t) + theta[1])*sin(alpha(t))**2*sin(theta[1])**2/r - T2*sin(alpha(t) + theta[1])*sin(alpha(t))**2*cos(theta[1])**2/r - T2*sin(alpha(t) + theta[1])*sin(theta[1])**2*cos(alpha(t))**2/r - T2*sin(alpha(

In [67]:
len(poly_dalamber)

3

In [69]:
# fuf, lul = symbols('fuf, lul')
# mum = Poly(fuf*2 + 4*lul + 3, [fuf, lul])
# len(mum.coeffs())

3

In [5]:
s_dx = read_obj('dalamber_dx_coeff')
s_dy = read_obj('dalamber_dy_coeff')
s_dalpha = read_obj('dalamber_dalpha_coeff')

In [7]:
print('-----\n', s_dx)
print('-----\n', s_dy)
print('-----\n', s_dalpha)

-----
 -delta_x*(3*d*m1*r*(nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) + nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) + sin(alpha(t))*Derivative(nu2(t), t) - cos(alpha(t))*Derivative(nu1(t), t)) + d*(T1*cos(alpha(t) + theta[0]) + T2*cos(alpha(t) + theta[1]) + T3*cos(alpha(t) + theta[2])) + r*(-W1*sin(alpha(t) + theta[0]) - W2*sin(alpha(t) + theta[1]) - W3*sin(alpha(t) + theta[2]) + b*sin(alpha(t) + theta[0])*Derivative(alpha(t), (t, 2)) + b*sin(alpha(t) + theta[1])*Derivative(alpha(t), (t, 2)) + b*sin(alpha(t) + theta[2])*Derivative(alpha(t), (t, 2)) + m2*(d + sin(alpha(t) + beta[0])*sin(alpha(t) + theta[0]))*(d*(Derivative(alpha(t), t) + Derivative(theta[0], t))*cos(alpha(t) + theta[0])*Derivative(alpha(t), t) + d*sin(alpha(t) + theta[0])*Derivative(alpha(t), (t, 2)) + nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) + nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) + sin(alpha(t) + beta[0])*Derivative(alpha(t), (t, 2)) + sin(alpha(t))*Derivative(nu2(t), t) + cos(alpha(t) + beta[0])*Derivat

In [74]:
coeff[delta['x']] = s_dx
coeff[delta['y']] = s_dy
coeff[delta['alpha']] = s_dalpha

In [75]:
# %%time

res_poly_dx = Poly(coeff[delta['x']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
res_poly_dy = Poly(coeff[delta['y']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
res_poly_dalpha = Poly(coeff[delta['alpha']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])


In [26]:
# %%time
# res_poly_dx = Poly(coeff[delta['x']], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])

In [27]:
# CHECK YOURSELF

# print(res_poly_dx)
# print(pa(res_poly_dy))
print(res_poly_dalpha)

Poly((-4*m2*sin(beta[0]) - 4*m2*sin(beta[1]) - 4*m2*sin(beta[2]))*Derivative(nu1(t), t) + (4*m2*cos(beta[0]) + 4*m2*cos(beta[1]) + 4*m2*cos(beta[2]))*Derivative(nu2(t), t) + (3*a + 6*b + 4*d*m2*cos(beta[0] - theta[0]) + 4*d*m2*cos(beta[1] - theta[1]) + 4*d*m2*cos(beta[2] - theta[2]) + 12*m2)*Derivative(alpha(t), (t, 2)) + 3*W + 6*d*m2*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[1] - theta[1])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[2] - theta[2])*Derivative(alpha(t), t)**2 + 2*m2*nu1(t)*sin(beta[0] - theta[0])*sin(theta[0])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[1] - theta[1])*sin(theta[1])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[2] - theta[2])*sin(theta[2])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[0])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[1])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[2])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[0])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[1])*Derivative(alpha(t), t) + 2*m2*nu

In [28]:
print(pa(res_poly_dx))
print(pa(res_poly_dy))
print(pa(res_poly_dalpha))

{Derivative(alpha(t), t), theta[2], beta[0], d, m2, alpha(t), beta[1], Derivative(alpha(t), (t, 2)), theta[0], m1, beta, beta[2], theta[1], theta, t}
None
{Derivative(alpha(t), t), theta[2], beta[0], d, m2, alpha(t), beta[1], Derivative(alpha(t), (t, 2)), theta[0], m1, beta, beta[2], theta[1], theta, t}
None
{Derivative(alpha(t), t), theta[2], a, theta[0], beta[2], theta[1], beta[0], d, W, alpha(t), beta[1], b, Derivative(alpha(t), (t, 2)), beta, m2, theta, t}
None


In [76]:
# CHECK YOURSELF

print(len(res_poly_dx.coeffs()))
print(len(res_poly_dy.coeffs()))
print(len(res_poly_dalpha.coeffs()))

4
4
4


In [56]:
res_poly_dx

Poly((3*m1*cos(alpha(t)) + 6*m2*cos(alpha(t)))*Derivative(nu1(t), t) + (-3*m1*sin(alpha(t)) - 6*m2*sin(alpha(t)))*Derivative(nu2(t), t) + (-2*d*m2*sin(alpha(t) + theta[0]) - 2*d*m2*sin(alpha(t) + theta[1]) - 2*d*m2*sin(alpha(t) + theta[2]) - 2*m2*sin(alpha(t) + beta[0]) - 2*m2*sin(alpha(t) + beta[1]) - 2*m2*sin(alpha(t) + beta[2]))*Derivative(alpha(t), (t, 2)) - 2*d*m2*cos(alpha(t) + theta[0])*Derivative(alpha(t), t)**2 - 2*d*m2*cos(alpha(t) + theta[1])*Derivative(alpha(t), t)**2 - 2*d*m2*cos(alpha(t) + theta[2])*Derivative(alpha(t), t)**2 - 3*m1*nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - 3*m1*nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu1(t)*sin(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu2(t)*cos(alpha(t))*Derivative(alpha(t), t) + 2*m2*sin(alpha(t) + theta[0])*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + 2*m2*sin(alpha(t) + theta[1])*sin(beta[1] - theta[1])*Derivative(alpha(t), t)**2 + 2*m2*sin(alpha(t) + theta[2])*sin(beta[2] - theta[2])*Derivative(alpha(t), t)*

In [57]:
res_poly_dy

Poly((3*m1*sin(alpha(t)) + 6*m2*sin(alpha(t)))*Derivative(nu1(t), t) + (3*m1*cos(alpha(t)) + 6*m2*cos(alpha(t)))*Derivative(nu2(t), t) + (2*d*m2*cos(alpha(t) + theta[0]) + 2*d*m2*cos(alpha(t) + theta[1]) + 2*d*m2*cos(alpha(t) + theta[2]) + 2*m2*cos(alpha(t) + beta[0]) + 2*m2*cos(alpha(t) + beta[1]) + 2*m2*cos(alpha(t) + beta[2]))*Derivative(alpha(t), (t, 2)) - 2*d*m2*sin(alpha(t) + theta[0])*Derivative(alpha(t), t)**2 - 2*d*m2*sin(alpha(t) + theta[1])*Derivative(alpha(t), t)**2 - 2*d*m2*sin(alpha(t) + theta[2])*Derivative(alpha(t), t)**2 + 3*m1*nu1(t)*cos(alpha(t))*Derivative(alpha(t), t) - 3*m1*nu2(t)*sin(alpha(t))*Derivative(alpha(t), t) + 6*m2*nu1(t)*cos(alpha(t))*Derivative(alpha(t), t) - 6*m2*nu2(t)*sin(alpha(t))*Derivative(alpha(t), t) - 2*m2*sin(alpha(t) + beta[0])*Derivative(alpha(t), t)**2 - 2*m2*sin(alpha(t) + beta[1])*Derivative(alpha(t), t)**2 - 2*m2*sin(alpha(t) + beta[2])*Derivative(alpha(t), t)**2 - 2*m2*sin(beta[0] - theta[0])*cos(alpha(t) + theta[0])*Derivative(alpha(t

In [58]:
res_poly_dalpha

Poly((-4*m2*sin(beta[0]) - 4*m2*sin(beta[1]) - 4*m2*sin(beta[2]))*Derivative(nu1(t), t) + (4*m2*cos(beta[0]) + 4*m2*cos(beta[1]) + 4*m2*cos(beta[2]))*Derivative(nu2(t), t) + (3*a + 6*b + 4*d*m2*cos(beta[0] - theta[0]) + 4*d*m2*cos(beta[1] - theta[1]) + 4*d*m2*cos(beta[2] - theta[2]) + 12*m2)*Derivative(alpha(t), (t, 2)) + 6*d*m2*sin(beta[0] - theta[0])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[1] - theta[1])*Derivative(alpha(t), t)**2 + 6*d*m2*sin(beta[2] - theta[2])*Derivative(alpha(t), t)**2 + 2*m2*nu1(t)*sin(beta[0] - theta[0])*sin(theta[0])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[1] - theta[1])*sin(theta[1])*Derivative(alpha(t), t) + 2*m2*nu1(t)*sin(beta[2] - theta[2])*sin(theta[2])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[0])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[1])*Derivative(alpha(t), t) + 4*m2*nu1(t)*cos(beta[2])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[0])*Derivative(alpha(t), t) + 2*m2*nu2(t)*sin(beta[1])*Derivative(alpha(t), t) + 2*m2*nu2(t)*s

In [77]:
# система уравнений left*[nu1', nu2', alpha''] = right

left = Matrix([res_poly_dx.coeffs()[:3],
               res_poly_dy.coeffs()[:3],
               res_poly_dalpha.coeffs()[:3]])
right = Matrix([res_poly_dx.coeffs()[-1],
                res_poly_dy.coeffs()[-1],
                res_poly_dalpha.coeffs()[-1]])

In [80]:
left_inv = left.inv()

KeyboardInterrupt: 

In [ ]:
# %%time

res = left_inv * right

In [ ]:
write_obj(res, 'vector_res_diff_eq_for', 'Вектор правых частей дифф уравнений на diff(nu1), diff(nu2), diff2(alpha)')

In [68]:
pa(res[0]) # nu1'

{beta[1], W, beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[2], t, W[1], alpha(t), Derivative(alpha(t), t), W[0]}


In [69]:
pa(res[1]) # nu2'

{beta[1], beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[1], W[2], t, W, alpha(t), Derivative(alpha(t), t), W[0]}


In [70]:
pa(res[2]) # alpha''

{beta[1], W, beta[0], beta, b, theta[1], d, m2, beta[2], theta[0], theta, m1, theta[2], a, W[2], t, W[1], alpha(t), Derivative(alpha(t), t), W[0]}


In [ ]:
%%time

simple_res = simplify(res[0])

In [ ]:
simplify(utils.subs_init(res))

In [ ]:
# ____________________________________________END___________________________________________

In [ ]:
# Полный Д'Аламбер Лагранжа

# A_full = 0
# A_full  = lambda i: A['platform'](i) + A['fork'](i) + A['wheel'](i)
# A_full_poly = lambda i: Poly(A_full(i).subs(delta['psi'][i], eq['diff(psi)'](i)).subs(delta['theta'][i], eq['diff(theta)'](i)), 
#     [delta['x'], delta['y'], delta['alpha']])

In [ ]:
# Полный Д'Аламбер для всего тела, просуммированный 

A_full_poly_sum = 0
for k in range(3):
    A_full_poly_sum += A_full_poly(k)

print_all_variables(A_full_poly_sum)

In [ ]:
print_all_variables(A_full_poly_sum)

In [ ]:
# Коэффиценты
# A_coeffs = lambda i: A_full_poly(i).coeffs()

In [ ]:
# Здесь будут лежать коэффиценты при dx dy dalpha  для всех 3 колёс
# ПОКА ДЛЯ ОДНОГО КОЛЕСА
#var = A_coeffs(0)

# coeff_wheel.append(A_coeffs(1))
# coeff_wheel.append(A_coeffs(2))

In [ ]:
#l = 0
#var[0] = var[0].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[1] = var[1].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0})
#var[2] = var[2].subs({alpha: 0, beta[l]: 0, theta[l]: 0, nu[1]: 0, nu[2]: 0}) 

In [ ]:
coeff[delta['x']]     = lambda i: simplify(A_coeffs(i)[0])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['y']]     = lambda i: simplify(A_coeffs(i)[1])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})
coeff[delta['alpha']] = lambda i: simplify(A_coeffs(i)[2])#.subs({alpha: 0, beta[i]: 0, theta[i]: 0, nu[1]: 0, nu[2]: 0})

In [ ]:
coeff_dx ={}
coeff_dy ={}
coeff_dalpha ={}

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[0] = coeff[delta['x']](0)
coeff_dy[0] = coeff[delta['y']](0)
coeff_dalpha[0] = coeff[delta['alpha']](0)

In [ ]:
coeff_dalpha[0]

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[1] = coeff[delta['x']](1)
coeff_dy[1] = coeff[delta['y']](1)
coeff_dalpha[1] = coeff[delta['alpha']](1)

In [ ]:
%%time
#коэффиценты для колеса при dx dy dalpha, просто в отдельные переменные
coeff_dx[2] = coeff[delta['x']](2)
coeff_dy[2] = coeff[delta['y']](2)
coeff_dalpha[2] = coeff[delta['alpha']](2)

In [ ]:
# надеюсь нормально их суммировать сейчас
coeff_dx['sum'] = 0
coeff_dy['sum'] = 0
coeff_dalpha['sum'] = 0

for k in range(3):
    coeff_dx['sum'] += coeff_dx[k]
    coeff_dy['sum'] += coeff_dy[k]
    coeff_dalpha['sum'] += coeff_dalpha[k]

In [ ]:
# говорим что получившиеся выражения это полином, зависящий от nu1, nu2, (alpha)'' 
eq_by_dx     = Poly(coeff_dx[0],     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dy     = Poly(coeff_dy[0],     [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])
eq_by_dalpha = Poly(coeff_dalpha[0], [Derivative(nu[1],t), Derivative(nu[2],t), Derivative(alpha,t,2)])

In [ ]:
len(eq_by_dx.coeffs())

In [ ]:
len(eq_by_dy.coeffs())

In [ ]:
# посмотреть что получилось
len(eq_by_dx.coeffs())

In [ ]:
# Пример группировки
print(collect(eq_by_dx,
        [Derivative(nu[1], t), Derivative(nu[2], t), Derivative(alpha, t,2)]))

In [ ]:
# система уравнений left*[nu1', nu2', alpha''] = right

left = Matrix([eq_by_dx.coeffs()[:3],
               eq_by_dy.coeffs()[:3],
               eq_by_dalpha.coeffs()[:3]])
right = Matrix([eq_by_dx.coeffs()[-1],
                eq_by_dy.coeffs()[-1],
                eq_by_dalpha.coeffs()[-1]])

In [ ]:
alpha_t, nu1_t, nu2_t = symbols('alpha_t, nu1_t, nu2_t')
theta0, theta1, theta2 = symbols('theta0, theta1, theta2')
alpha1, alpha2 = symbols('alpha1, alpha2')

In [ ]:
def subs_for_ode(eq):
    eq = eq.subs({
            nu[1]: nu1_t, nu[2]: nu2_t,
            theta[0]: theta0, theta[1]: theta1, theta[2]: theta2,
            alpha: alpha1, Derivative(alpha, t): alpha2
    })
    return eq

In [ ]:
a = []
a += A_full_poly_sum.find(alpha)
a


In [ ]:
left = subs_init(left)
right = subs_init(right)

In [ ]:
print(left)

In [ ]:
%%time

left_inv = left.inv() # left^(-1)

In [ ]:
# Какие то варианты, которые наверно делают тоже самое
# from sympy.solvers.solveset import linsolve
# from sympy.solvers.solvers import solve_linear_system_LU

In [ ]:
eq['diff_eq'] = left_inv*right

In [ ]:
len(eq['diff_eq']) # nu1', nu2', alpha''

In [ ]:
eq['diff_eq']

In [ ]:
len(eq['diff_eq'])

In [ ]:
eq['psi']   = []
eq['theta'] = []

for k in range(3):
    eq['theta'].append(subs_init(eq['diff(theta)_nu'](k)))
    eq['psi'].append(subs_init(eq['diff(psi)_nu'](k)))

In [ ]:
# уравнения с 6 по 10

for k in range(3):
    eq['theta'][k] = subs_for_ode(eq['theta'][k])
    eq['psi'][k] = subs_for_ode(eq['psi'][k])
    
#####################
# константы  подставлены
#####################

In [ ]:
eq['psi'][0]

In [ ]:
for k in range(3):
    eq['diff_eq'] = eq['diff_eq'].subs({
        Derivative(theta[k], t): eq['theta'][k]
    })

In [ ]:
# уравнения с 1 по 3, с подставленными theta[i]'
T0, W0 = symbols('T0, W0')
eq['diff_eq'] = subs_for_ode(eq['diff_eq']).subs({
    W[0]: W0,
    T[0]: T0
})[2]

In [ ]:
dict_eq = {}
list_eq = [0,0,0,0,0,0,0,0,0,0]

list_eq[0] = dict_eq['nu1'] = eq['diff_eq'][0]
list_eq[1] = dict_eq['nu2'] = eq['diff_eq'][1]
list_eq[2] = dict_eq['alpha2'] = eq['diff_eq'][2]
list_eq[3] = dict_eq['alpha1'] = alpha2
list_eq[4] = dict_eq['theta0'] = eq['theta'][0]
list_eq[5] = dict_eq['theta1'] = eq['theta'][1]
list_eq[6] = dict_eq['theta2'] = eq['theta'][2]
list_eq[7] = dict_eq['psi0'] = eq['psi'][0]
list_eq[8] = dict_eq['psi1'] = eq['psi'][1]
list_eq[9] = dict_eq['psi2'] = eq['psi'][2]

In [ ]:
from mpmath import *

mp.dps = 15;  # точность
mp.pretty = True

In [ ]:
list_eq[0]

In [ ]:
temp = lambdify([nu[1], nu[2], alpha1, alpha2, theta0, theta1, theta2, psi[0], psi[1], psi[2]], list_eq[0])

In [ ]:
#############################################
# Далее решение этой системы дифф уравнений 
#############################################

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from math import *

In [ ]:
type(list_eq[0])

In [ ]:
def f(y, t, T0, W0, d):
    '''Определение правой части системы лин уравнений'''
    nu1_t, nu2_t, alpha1, alpha2, theta0, theta1, theta2, psi0, psi1, psi2 = y
    return list_eq

In [ ]:
begin_v = [0,0,0,0,0,0,0,0,0,0]
t = np.linspace(0, 1, 51) # ну вот такое себе конечно
T0 = 1.0
W0 = 1.0
d = 0.5

In [ ]:
res = odeint(f, begin_v, t, args=(T0, W0, d,))

In [ ]:
list_res = []

for x in range(10):
    list_res.append(res[:,x])